<a href="https://colab.research.google.com/github/makkimaki/metric-learning/blob/main/image_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd 
import torch
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
import time
from typing import List, Optional, Tuple
import faiss


In [ ]:
class ImageRetrieval(object):
    def __init__(self,
                 feature_extractor, 
                 features_path: Path,
                 load_index: bool = False,
                 index_path: str = None,
                 use_gpu: bool = False
                 ):
        features = np.load(features_path)
        self._features = torch.from_numpy(features["images_reduced"])
        self._extractor = feature_extractor
        self._use_gpu = use_gpu
    
    def retrieve(
        self,
        image: np.ndarray,
        top_k: int = 8,
        ) -> Tuple[np.ndarray, np.ndarray]:

        features = self._extractor.extract(image)



SyntaxError: ignored

In [ ]:
def remove_negative_index(index_vector: np.ndarray) -> np.ndarray:
    """Non-negative index extractor

    Args: 
        index_vector (np.ndarray): 1d vector indices
    
    Return:
        np.ndarray: 1d vector index without '-1' indices

    """
    array_indices = np.delete(index_vector, np.where(index_vector<0)[0], axis=0)

    return array_indices


class FaissHNSW(object):
    def __init__(self, load_index, index_path, param_dict, features):
        if not load_index:
            dim = param_dict["dim"]
            M = param["M"]

            self._index = faiss.IndexHNSWFlat(dim, M, faiss.METRIC_INNER_PRODUCT)
            self._index.add(features)

            ## save indices
            faiss.write_index(self._index, index_path)
            print(f"Faiss index is saved to: {index_path}")

        else:
            # load index
            self._index = faiss.read_index(index_path)
            print(f"FaissHNSW index loaded from: {index_path}")

    def search(self, query_vector, top_k):
        similarity, indices = self._index.search(query_vector, top_k)
        similarity, indices = similarity[0], indices[0]
        indices = remove_negative_index(indices)

        return indices, similarity

